# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,11.84,69,33,6.71,CA,1693473085
1,1,neiafu,-18.6500,-173.9833,23.69,75,69,10.27,TO,1693473088
2,2,waitangi,-43.9535,-176.5597,9.90,85,0,0.45,NZ,1693473093
3,3,grytviken,-54.2811,-36.5092,-1.90,89,100,1.69,GS,1693473095
4,4,tiksi,71.6872,128.8694,1.07,74,100,8.22,RU,1693473100


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot

map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5

)
# Display the map
map_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_wanted=city_data_df.loc[(city_data_df["Max Temp"]<24)&(city_data_df["Max Temp"]>21)
                             &(city_data_df["Wind Speed"]<3)&(city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
city_wanted.dropna()

# Display sample data
city_wanted.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,55,newman,37.3138,-121.0208,21.27,31,0,0.90,US,1693473231
144,144,matoury,4.8483,-52.3317,22.98,100,0,0.00,GF,1693473279
267,267,amga,60.8953,131.9608,22.82,58,0,1.64,RU,1693473783
357,357,patterson,37.4716,-121.1297,23.21,28,0,0.90,US,1693474017
411,411,dana point,33.4670,-117.6981,22.60,94,0,2.24,US,1693474154


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_wanted[["City", "Country","Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, 'Hotel Name', "")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
55,newman,US,37.3138,-121.0208,31,
144,matoury,GF,4.8483,-52.3317,100,
267,amga,RU,60.8953,131.9608,58,
357,patterson,US,37.4716,-121.1297,28,
411,dana point,US,33.4670,-117.6981,94,
424,palapye,BW,-22.5460,27.1251,33,
450,remire-montjoly,GF,4.9167,-52.2667,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {}
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: No hotel found
matoury - nearest hotel: La Marmotte
amga - nearest hotel: "АммА"
patterson - nearest hotel: Hampton Inn & Suites
dana point - nearest hotel: Laguna Cliffs Marriott Resort & Spa
palapye - nearest hotel: Palapaye Hotel
remire-montjoly - nearest hotel: Complexe Belova


,City,Country,Lat,Lng,Humidity,Hotel Name
55,newman,US,37.3138,-121.0208,31,No hotel found
144,matoury,GF,4.8483,-52.3317,100,La Marmotte
267,amga,RU,60.8953,131.9608,58,"""АммА"""
357,patterson,US,37.4716,-121.1297,28,Hampton Inn & Suites
411,dana point,US,33.4670,-117.6981,94,Laguna Cliffs Marriott Resort & Spa
424,palapye,BW,-22.5460,27.1251,33,Palapaye Hotel
450,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = 0.5,
    size = "Humidity",
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
map_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)